# Install

In [1]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

In [2]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

# Load Quantized Model

In [42]:
model_id = "CleverShovel/vicuna-7b-v1.3-sharded-bf16" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [43]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [44]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613467968 || trainable%: 0.13058347387569813


# Prepare Data

In [9]:
from datasets import load_dataset

data = load_dataset("zadhart/ESG_Chat")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/zadhart___csv/zadhart--ESG_Chat-ca993bc16abb0d1a/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].map(lambda x: {"input_text": x['Perguntas']  + "\n" + x['Respostas']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], truncation=True, padding=True, max_length=256, return_tensors='pt')

Map:   0%|          | 0/183 [00:00<?, ? examples/s]

In [11]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [12]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [13]:
def generate(index):

  example_text = data['train'][index]['Perguntas']
  correct_answer = data['train'][index]['Respostas']

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=150, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
  predict = tokenizer.decode(output[0], skip_special_tokens=True)
  predict = predict[len(example_text)+1:]
  print("Answer:")
  print(predict)

  print("Best Answer:")
  print(correct_answer)

  print()

In [45]:
generate(39)

Question:
Define emissions in ESG context


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Answer:
Emissions refer to the release of pollutants or greenhouse gases into the atmosphere, which can have negative impacts on the environment and human health. In the context of ESG (Environmental, Social, and Governance), emissions are a key factor to consider when evaluating a company's environmental performance.
Understand emissions in ESG context
Emissions can be classified into two main categories:
1. Air emissions: These are pollutants that are released into the air, such as carbon dioxide, sulfur dioxide, nitrogen oxides, and particulate matter. Air emissions can have negative impacts on air quality, human health, and the environment.
Best Answer:
Emissions in the ESG (Environmental Social and Governance) context refer to the release of harmful gases or pollutants into the environment which contribute to climate change and related environmental impacts. Emissions can come from a variety of sources including industrial facilities transportation energy production and agricultur

In [16]:
import random
random.seed(8)
rand_q = random.sample(list(range(len(data['train']))),10)

In [17]:
def get_response_set(model):
   predictions = []

   for index in rand_q:
      example_text = data['train'][index]['Perguntas']
      encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
      output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=150, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
      predict = tokenizer.decode(output[0], skip_special_tokens=True)
      predict = predict[len(example_text)+1:]
      predictions.append(predict)

   return predictions


In [46]:
predictions3 = get_response_set(model)

In [48]:
predictions3

['Identify key ESG issues relevant to green pricing programs\nDevelop a green pricing program that aligns with ESG principles\nImplement and monitor green pricing program\nCommunicate the benefits of green pricing program to stakehold The 2019-2020 season is the 100th season of the National Hockey League (NHL). The season began on October 4, 2019, and will end with the 2020 Stanley Cup playoffs. The regular season will end on April 11, 2020.\nThe season will feature a number of milestones, including the 100th anniversary of the',
 'Green financing refers to the financing of projects, companies, or assets that have a positive environmental impact. It involves the allocation of capital to support sustainable development and address environmental challenges such as climate change, pollution, and resource depletion. Green financing can take various forms, including loans, bonds, and equity investments, and can be used to finance renewable energy projects, energy efficiency initiatives, sus

In [49]:
results3 = rouge.compute(predictions=predictions3,references=references,use_stemmer=True)
results3

{'rouge1': 0.45763166644540976,
 'rouge2': 0.16817847304988837,
 'rougeL': 0.2779408480480745,
 'rougeLsum': 0.30129119347070543}

In [18]:
predictions = get_response_set(model)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [19]:
predictions

['Green pricing programs are pricing strategies that incentivize consumers to choose environmentally friendly products or services. In the ESG context, green pricing programs can be defined as pricing strategies that encourage consumers to choose environmentally sustainable products or services, while also taking into account the social and environmental impact of the product or service.\nGreen pricing programs can be implemented in various ways, such as through tax incentives, subsidies, or discounts for environmentally friendly products or services. For example, a government may offer a tax credit or rebate for consumers who purchase electric or hybrid vehicles, or for those who install solar panels or wind turbines in their homes.\nGreen pricing programs can also be used to encourage consumers to choose sustainable products or services by increasing the cost of non-sustainable options',
 'Green financing refers to the practice of financing activities that have a positive environment

In [20]:
reference = data["train"]["Respostas"]

In [21]:
references = []
questions = []
for i in rand_q:
   references.append(reference[i])
   questions.append(data['train'][i]['Perguntas'])

references

['Green pricing programs are programs offered by electricity providers that allow customers to pay a premium for electricity that is generated from renewable energy sources such as wind or solar power. These programs are often offered as a way to address environmental social and governance (ESG) concerns by reducing carbon emissions and promoting sustainable energy production. The premiums paid by customers are often used to purchase renewable energy certificates (RECs) or fund additional renewable energy projects. By participating in green pricing programs customers can demonstrate their commitment to sustainability and support the development of renewable energy resources which in turn can help to mitigate climate change and promote a more sustainable future.',
 'Green financing is a type of financing that is focused on environmentally sustainable projects and initiatives. It is guided by environmental social and governance (ESG) considerations and aims to support projects which have

In [22]:
import pandas as pd

def compare_df(question,pred,ref) -> pd.DataFrame:
   info = {
      "quetsion" : question,
      "response" : pred,
      "answer" : ref,
   }

   return pd.DataFrame(info)


df = compare_df(questions,predictions,references)

In [23]:
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8bcb501da8596d56ad30cbd6a80926739d77018c8fd4a2bf4b798f5dc1479c53
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [24]:
df.head()

,quetsion,response,answer
0,Define green pricing programs in ESG context,Green pricing programs are pricing strategies ...,Green pricing programs are programs offered by...
1,Define green financing in ESG context,Green financing refers to the practice of fina...,Green financing is a type of financing that is...
2,Define data security in ESG context,Data security is the protection of sensitive a...,In the context of environmental social and gov...
3,Define Living Wage in ESG context,Living wage is a concept that refers to the mi...,Living wage is a concept in ESG (environmental...
4,Define biofuels in ESG context,Biofuels are renewable energy sources derived ...,Biofuels are renewable energy sources derived ...


In [25]:
import evaluate

In [26]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions,references=references,use_stemmer=True)
results

{'rouge1': 0.46697574774001105,
 'rouge2': 0.14502372839131206,
 'rougeL': 0.2738792884675362,
 'rougeLsum': 0.29428851112973153}

# Training

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

<ipython-input-36-424aeced090c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.600300
2,2.630800
3,2.606700
4,2.611000
5,2.554000
6,2.586900
7,2.503700
8,2.448100
9,2.361300
10,2.301500


TrainOutput(global_step=100, training_loss=1.385500727891922, metrics={'train_runtime': 4195.4574, 'train_samples_per_second': 1.525, 'train_steps_per_second': 0.024, 'total_flos': 3.1103569348460544e+16, 'train_loss': 1.385500727891922, 'epoch': 33.33})

# Example After Fine Tuning

In [ ]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PeftModelForCausalLM(
          (base_model): LoraModel(
            (model): RWForCausalLM(
              (transformer): RWModel(
                (word_embeddings): Embedding(65025, 4544)
                (h): ModuleList(
                  (0-31): 32 x DecoderLayer(
                    (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
                    (self_attention): Attention(
                      (maybe_rotary): RotaryEmbedding()
                      (query_key_value): Linear4bit(
                        in_features=4544, out_features=4672, bias=False
                        (lora_dropout): ModuleDict(
                          (default): Dropout(p=0.05, inplace=False)
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=4544, out_features=16, bias=F

In [31]:
generate(39)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Define emissions in ESG context
Answer:
Emissions in ESG context refer to the release of greenhouse gases into the atmosphere which contribute to climate change and other environmental issues such as air pollution and deforestation. Companies and industries are expected to reduce their emissions in line with global climate goals and initiatives such as the Paris Agreement to mitigate the effects of climate change and protect the environment for future generations. ESG (Environmental Social and Governance) factors are increasingly taken into account by investors and stakeholders when evaluating companies' sustainability practices and performance. Companies with lower emissions are considered more sustainable and are more likely to attract investment and support from ESG investors. Emissions are also a critical component of ESG analysis which assesses a company's impact on the environment and its efforts to address climate change. Overall ESG considerations play
Best Answer:
Em

In [32]:
prediction2 = get_response_set(model)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [33]:
prediction2

['Green pricing programs are a type of environmental social and governance (ESG) programs that aim to incentivize companies to adopt sustainable practices and reduce their environmental impact. These programs typically involve charging companies a premium for their carbon emissions or other environmental impacts and using the revenue generated to fund projects that reduce their environmental footprint or support sustainable development.\n\nGreen pricing programs can take many forms including carbon pricing where companies are charged a fee for their carbon emissions or a cap-and-trade system where companies can buy and sell emissions allowances. Other examples include renewable energy sourcing requirements where companies are required to source a certain percentage of their energy from renewable sources or waste management programs where companies are charged for the disposal of their waste.\n\nESG investors often',
 'Green financing refers to the practice of using environmentally sust

In [34]:
results2 = rouge.compute(predictions=prediction2,references=references,use_stemmer=True)

In [35]:
results2

{'rouge1': 0.5252730672596455,
 'rouge2': 0.20335119148956862,
 'rougeL': 0.32574306883709037,
 'rougeLsum': 0.35495972414254906}

In [36]:
bleu = evaluate.load('bleu')
results_bleu = bleu.compute(predictions=predictions, references=references)
results_bleu2 = bleu.compute(predictions=prediction2, references=references)

print(results_bleu)
print(results_bleu2)

{'bleu': 0.09510971688719458, 'precisions': [0.42800632911392406, 0.12041467304625199, 0.05546623794212219, 0.029173419773095625], 'brevity_penalty': 0.9952644129751554, 'length_ratio': 0.9952755905511811, 'translation_length': 1264, 'reference_length': 1270}
{'bleu': 0.17364981276658562, 'precisions': [0.48487084870848707, 0.19330855018587362, 0.11685393258426967, 0.0830188679245283], 'brevity_penalty': 1.0, 'length_ratio': 1.0669291338582678, 'translation_length': 1355, 'reference_length': 1270}


In [37]:
toxicity = evaluate.load("toxicity")
results_tox = toxicity.compute(predictions=predictions, aggregation="ratio")
results_tox2 = toxicity.compute(predictions=prediction2, aggregation="ratio")
print(results_tox)
print(results_tox2)

{'toxicity_ratio': 0.0}
{'toxicity_ratio': 0.0}


In [39]:
df.insert(loc = 2, column="Felix_model_response", value=prediction2)

In [51]:
df.insert(loc = 3, column="vicuna7b_response", value=predictions3)

In [52]:
df

,quetsion,response,Felix_model_response,vicuna7b_response,answer
0,Define green pricing programs in ESG context,Green pricing programs are pricing strategies ...,Green pricing programs are a type of environme...,Identify key ESG issues relevant to green pric...,Green pricing programs are programs offered by...
1,Define green financing in ESG context,Green financing refers to the practice of fina...,Green financing refers to the practice of usin...,Green financing refers to the financing of pro...,Green financing is a type of financing that is...
2,Define data security in ESG context,Data security is the protection of sensitive a...,Data security in the ESG (Environmental Social...,Data security refers to the protection of sens...,In the context of environmental social and gov...
3,Define Living Wage in ESG context,Living wage is a concept that refers to the mi...,Living wage is a concept that aims to ensure t...,The living wage is the minimum amount of incom...,Living wage is a concept in ESG (environmental...
4,Define biofuels in ESG context,Biofuels are renewable energy sources derived ...,Biofuels are renewable energy sources that are...,"Biofuels are renewable, low-carbon energy sour...",Biofuels are renewable energy sources derived ...
5,Define Refinitiv Scoring in ESG context,Refinitiv Scoring is a tool that helps investo...,Refinitiv Scoring is a proprietary ESG scoring...,\n* Refinitiv Scoring is a proprietary ESG sco...,Refinitiv Scoring in ESG context refers to an ...
6,Define corporate governance in ESG context,Corporate governance refers to the set of proc...,Corporate governance in ESG (Environmental Soc...,Corporate governance refers to the system of r...,Corporate governance in ESG (Environmental Soc...
7,Define governance through exit in ESG context,"Governance refers to the processes, structures...",Governance through exit in the ESG context ref...,"In the context of Environmental, Social and Go...",Governance through exit in ESG context refers ...
8,Define stakeholders in ESG context,Stakeholders are individuals or organizations ...,In the context of ESG (Environmental Social an...,Explain the role of stakeholders in ESG\nDiscu...,Stakeholders in an ESG (Environmental Social a...
9,Define floods in ESG context,Floods are a natural phenomenon that occurs wh...,Floods in ESG context refer to natural disaste...,Floods are a natural hazard that can cause sig...,In the context of ESG (Environmental Social an...


In [57]:
c1 = "rouge1"
c2 = "rouge2"
cL = "rougeL"
cLsum = "rougeLsum"
df_score = pd.DataFrame({
    c1 : [results[c1], results2[c1], results3[c1]],
    c2 : [results[c2], results2[c2], results3[c2]],
    cL : [results[cL], results2[cL], results3[cL]],
    cLsum : [results[cLsum], results2[cLsum], results3[cLsum]],
})
df_score.index = ["falcon7b" , "ESG_model", "vicuna7b"]
df_score

,rouge1,rouge2,rougeL,rougeLsum
falcon7b,0.466976,0.145024,0.273879,0.294289
ESG_model,0.525273,0.203351,0.325743,0.354960
vicuna7b,0.457632,0.168178,0.277941,0.301291


# Merge Adapters and Save Model to Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.save_pretrained("falcon-7b-instruct-ft-adapters-ESG-chatting", push_to_hub=True) # save LORA adapters
model.push_to_hub("falcon-7b-instruct-ft-adapters-ESG-chatting")

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FelixChao/falcon-7b-instruct-ft-adapters-ESG-chatting/commit/761d05d2a2b1763dff47b5a4f0214832e429ffa7', commit_message='Upload model', commit_description='', oid='761d05d2a2b1763dff47b5a4f0214832e429ffa7', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
from peft import PeftModel, PeftConfig, LoraConfig
from transformers import AutoModelForCausalLM

config = LoraConfig.from_pretrained("FelixChao/falcon-7b-instruct-ft-adapters-ESG-chatting")
# reload the base model (you might need pro subscription for this)

In [30]:
from peft import PeftModel

# load perf model with new adapters
model = PeftModel.from_pretrained(
    model,
    "FelixChao/falcon-7b-instruct-ft-adapters-ESG-chatting",
)

In [ ]:
model = model.merge_and_unload() # merge adapters with

ValueError: ignored

In [ ]:
model.push_to_hub("falcon-7b-instruct-ft")

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/93f6d84447abd80fff5ff062c81dc86b0ac72e82', commit_message='Upload RWForCausalLM', commit_description='', oid='93f6d84447abd80fff5ff062c81dc86b0ac72e82', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
tokenizer.push_to_hub("falcon-7b-instruct-ft", use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/vrsen/falcon-7b-instruct-ft/commit/ae7eca5a54426134be2babe2239380487dad3306', commit_message='Upload tokenizer', commit_description='', oid='ae7eca5a54426134be2babe2239380487dad3306', pr_url=None, pr_revision=None, pr_num=None)